# MERRA2 Daily Means Granularity Test
- Download
- Trim
- Create Cutout
- Generate Temperature Output

## 1. Imports and Create Dataset

In [1]:
import calendar
import geodata
import xarray as xr
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)
DS_daily = geodata.Dataset(module="merra2",
					 years=slice(1998, 1998),
					 months=slice(1,1),
                     weather_data_config = "surface_flux_dailymeans")  


INFO:geodata.dataset:Directory /Users/williamhonaker/Desktop/davidson/data_for_geodata/merra2 found, checking for completeness.
INFO:geodata.dataset:Directory complete.


In [2]:
if DS_daily.prepared == False:
	DS_daily.get_data()

## 2. Trim

In [8]:
## Variables before trimming
ds = xr.open_dataset(DS_daily.downloadedFiles[0][1])
list(ds.data_vars)

['HOURNORAIN', 'T2MMAX', 'T2MMEAN', 'T2MMIN', 'TPRECMAX']

In [9]:
# Trim Variables
DS_daily.trim_variables()

In [10]:
## Variables after trimming
ds = xr.open_dataset(DS_daily.downloadedFiles[0][1])
list(ds.data_vars)

['t2mmean', 'tprecmax']

In [4]:
DS_daily.__dict__

{'module': 'merra2',
 'config': 'surface_flux_dailymeans',
 'dataset_module': <module 'geodata.datasets.merra2' from '/usr/local/lib/python3.7/site-packages/geodata-0.0.2-py3.7.egg/geodata/datasets/merra2.py'>,
 'weatherconfig': {'file_granularity': 'dailymeans',
  'tasks_func': <function geodata.datasets.merra2.tasks_daily_merra2(xs, ys, yearmonths, prepare_func, **meta_attrs)>,
  'meta_prepare_func': <function geodata.datasets.merra2.prepare_meta_merra2(xs, ys, year, month, template, module, **params)>,
  'prepare_func': <function geodata.datasets.merra2.prepare_dailymeans_surface_flux(fn, year, month, xs, ys)>,
  'template': '/Users/williamhonaker/Desktop/davidson/data_for_geodata/merra2/{year}/{month:0>2}/MERRA2_*.statD_2d_slv_Nx.*.nc4',
  'url': 'https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2/M2SDNXSLV.5.12.4/{year}/{month:0>2}/MERRA2_{spinup}.statD_2d_slv_Nx.{year}{month:0>2}{day:0>2}.nc4',
  'fn': '/Users/williamhonaker/Desktop/davidson/data_for_geodata/merra2/{year}/{mont

In [5]:
DS_daily.weatherconfig['variables'] 

['t2mmean', 'tprecmax']

## 3. Cutout

In [12]:
cutout = geodata.Cutout(name="tokyo-2008-daily-test-7",
                       module="merra2",
                       weather_data_config="surface_flux_dailymeans",
                       xs=slice(138.5, 139.5),
                       ys=slice(35, 36),
                       years=slice(1998, 1998),
                       months=slice(1,1))

INFO:geodata.cutout:All cutout (tokyo-2008-daily-test-7, /Users/williamhonaker/Desktop/davidson/data_for_geodata/cutouts) files available.
INFO:geodata.preparation:[4, 1, 1, 10]
INFO:geodata.cutout:Cutout subset prepared: <Cutout tokyo-2008-daily-test-7 x=138.75-139.38 y=35.00-36.00 time=1998/1-1998/1 prepared>


In [13]:
cutout.prepare()

INFO:geodata.preparation:The cutout is already prepared. If you want to recalculate it, supply an `overwrite=True` argument.


True

In [9]:
cutout.__dict__

{'name': 'tokyo-2008-daily-test-7',
 'cutout_dir': '/Users/williamhonaker/Desktop/davidson/data_for_geodata/cutouts/tokyo-2008-daily-test-7',
 'prepared': True,
 'meta_append': 0,
 'config': 'surface_flux_dailymeans',
 'meta': <xarray.Dataset>
 Dimensions:     (time: 31, x: 2, y: 3, year-month: 1)
 Coordinates:
   * x           (x) float64 138.8 139.4
   * time        (time) datetime64[ns] 1998-01-01 1998-01-02 ... 1998-01-31
   * y           (y) float64 35.0 35.5 36.0
     lon         (x) float64 ...
     lat         (y) float64 ...
   * year-month  (year-month) MultiIndex
   - year        (year-month) int64 1998
   - month       (year-month) int64 1
 Data variables:
     *empty*
 Attributes:
     nco_openmp_thread_number:          1
     Comment:                           GMAO filename: d5124_m2_jan91.statD_2d...
     History:                           Original file generated: Sun Nov 16 08...
     Filename:                          MERRA2_200.statD_2d_slv_Nx.19980123.nc4
     Conven

## 4.  Generate Outputs

In [15]:
## Only Temperature output possible for Merra2 Daily Means currently

ds_temperature = geodata.convert.temperature(cutout)
ds_temperature

In [16]:
df_temperature = ds_temperature.to_dataframe(name='temperature')
df_temperature

lat      lon  temperature
time                y    x                                  
1998-01-01 12:00:00 35.0 138.750  35.0  138.750     8.314087
                         139.375  35.0  139.375    10.843048
                    35.5 138.750  35.5  138.750     0.837891
                         139.375  35.5  139.375     4.513550
                    36.0 138.750  36.0  138.750    -0.043091
...                                ...      ...          ...
1998-01-31 12:00:00 35.0 139.375  35.0  139.375     7.731232
                    35.5 138.750  35.5  138.750    -3.248901
                         139.375  35.5  139.375     1.689819
                    36.0 138.750  36.0  138.750    -4.639099
                         139.375  36.0  139.375     0.393372

[186 rows x 3 columns]

In [ ]:
df_temperature.to_csv('temperature_test_dailymeans.csv')